In [8]:
import os
import glob
from glob import glob as gg

from collections import defaultdict

import re
import sqlite3

import pandas as pd
from pandas import DataFrame
from pandas import Series

import numpy as np
from numpy import random, asarray, linspace, corrcoef
from datetime import datetime, date, time

import jieba
import jieba.posseg as pseg
import jieba.analyse
from optparse import OptionParser
from __future__ import print_function, unicode_literals
import sys

get_ipython().magic(u'pwd')
get_ipython().magic(u'cd "/Users/weiaiwaynexu/Dropbox/Acer Laptop Sync/Data Science/OC"')

con = sqlite3.connect("OccupyHK.sqlite")
cur = con.cursor()
df = pd.read_sql_query("SELECT * FROM Posts", con)
con.close()
df['published_date'] = pd.to_datetime(df['published_date'])
df = df.sort_values(by='published_date')
df['share_count'] = pd.to_numeric(df['share_count'])
df = df.fillna(0)
df = df[(df['published_date'] >= '2014-09-01') & (df['published_date'] <= "2014-12-30")]
print ("the earliest date:", df['published_date'].head(1), "the latest date:", df['published_date'].tail(1))

yellow_official = df.loc[df['org_name'].isin(['學民思潮 Scholarism',
                                                 'Occupy Central 佔領中環', 
                                                 '讓愛與和平佔領中環 Occupy Central with Love and Peace', 
                                                 '926 平民在政總現場','佔領期間 民間智慧選輯', '傘下爸媽',
                                                 '香港天主教正義和平委員會','Hong Kong Democracy Now', '踢爆五毛','香港警權關注組',
                                                 '維護香港核心價值', '我係香港人', '香港人 Secrets','保衛香港自由聯盟','9反佔中 HK oic',
                                                 'Fluid Occupiers'])] 


blue_official = df.loc[df['org_name'].isin(['幫港出聲 Silent Majority','Speak Out HK 港人講地','藍絲帶運動 The Blue Ribbon Movement','向香港警察致敬',
                                               '反佔中苦主大聯盟','泛民不代表我','聲討黃之鋒',
                                               '一人一Like救學生，反對中學生衝擊警察及佔中！',
                                               '「保普選 反佔中」簽左名揮手區！',
                                               '我要真普洱行動','我撐警務處長曾偉雄！',
                                               '我係大學生，我愛國愛港'])] 


print ("There are",len(yellow_official), "OFFICIAL posts sent by anti-establishment civil groups")
print ("There are",len(blue_official), "OFFICIAL posts sent by pro-establishment civil groups")

/Users/weiaiwaynexu/Dropbox/Acer Laptop Sync/Data Science/OC
the earliest date: 8635   2014-09-01
Name: published_date, dtype: datetime64[ns] the latest date: 34084   2014-12-24 23:00:01
Name: published_date, dtype: datetime64[ns]
There are 23884 OFFICIAL posts sent by anti-establishment civil groups
There are 6477 OFFICIAL posts sent by pro-establishment civil groups


In [55]:
yellow_official_top = yellow_official[yellow_official['share_count'] >= yellow_official["share_count"].quantile(0.75)]
yellow_official_bottom = yellow_official[yellow_official['share_count'] == 0]
blue_official_top = blue_official[blue_official['share_count'] >= blue_official["share_count"].quantile(0.75)]
blue_official_bottom = blue_official[blue_official['share_count'] == 0]

In [56]:
def convert2corpus(txt):
    corpus = []
    for index, i in txt.iterrows():
        corpus.append(i['content'])
    corpus = str(corpus)
    corpus = corpus.replace("[" and "]" and "\\", " ")
    return corpus

def cnpuncttoen(corpus):
    #corpus_new = corpus.replace(".", "")
    corpus_new = corpus.replace("。", "*.")
    corpus_new = corpus_new.replace("？", "*?")
    corpus_new = corpus_new.replace(",", "*,")
    corpus_new = corpus_new.replace("！", "*!")
    return corpus_new

def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('*.'):
        if '*?' in s_str:
            list_ret.extend(s_str.split('*?'))
        elif '*!' in s_str:
            list_ret.extend(s_str.split('*!'))
        else:
            list_ret.append(s_str)
    return list_ret

yellow_official_top_corpus = sentence_split(cnpuncttoen(convert2corpus(yellow_official_top))) 
yellow_official_bottom_corpus = sentence_split(cnpuncttoen(convert2corpus(yellow_official_bottom))) 
blue_official_top_corpus = sentence_split(cnpuncttoen(convert2corpus(blue_official_top)))
blue_official_bottom_corpus = sentence_split(cnpuncttoen(convert2corpus(blue_official_bottom))) 

In [57]:
jieba.analyse.set_stop_words('cn_stopword.txt')

def cracut(txt):
    cuttxt1 = [[(j.word, j.flag) for j in jieba.posseg.cut(i)] for i in txt]
    cracut = [[token for token, tag in sent if re.match('n|a', tag) and len(token)>1] 
                for sent in cuttxt1]
    return cracut

def cut(txt):
    cuttxt2 = [[(j.word, j.flag) for j in jieba.posseg.cut(i)] for i in txt]
    cut = [[token for token, tag in sent if len(token)>1] 
                for sent in cuttxt2]
    return cut

yellow_official_top_cra = cracut(yellow_official_top_corpus)
yellow_official_bottom_cra = cracut(yellow_official_bottom_corpus)
blue_official_top_cra = cracut(blue_official_top_corpus)
blue_official_bottom_cra = cracut(blue_official_bottom_corpus)

yellow_official_top_cut = cut(yellow_official_top_corpus)
yellow_official_bottom_cut = cut(yellow_official_bottom_corpus)
blue_official_top_cut = cut(blue_official_top_corpus)
blue_official_bottom_cut = cut(blue_official_bottom_corpus)

In [58]:
import itertools as it 
import networkx as nx

def semanticnetwork(inputlist):
    edgelist = [edge for phrase in inputlist for edge in it.combinations(phrase, 2)]
    word_network = nx.Graph(edgelist)
    return word_network

yellow_official_top_nx = semanticnetwork(yellow_official_top_cra)
yellow_official_bottom_nx = semanticnetwork(yellow_official_bottom_cra)
blue_official_top_nx = semanticnetwork(blue_official_top_cra)
blue_official_bottom_nx = semanticnetwork(blue_official_bottom_cra)

In [59]:
nx.write_gpickle(yellow_official_top_nx, "ARNOVA Analysis/yellow_official_top_nx_during.pickle")
nx.write_gpickle(yellow_official_bottom_nx, "ARNOVA Analysis/yellow_official_bottom_nx_during.pickle")
nx.write_gpickle(blue_official_top_nx, "ARNOVA Analysis/blue_official_top_nx_during.pickle")
nx.write_gpickle(blue_official_bottom_nx, "ARNOVA Analysis/blue_official_bottom_nx_during.pickle")
#nx.write_graphml(xxxx, "ARNOVA Analysis/test_cra_nx.graphml")


In [60]:
yellow_official_top_bt = nx.betweenness_centrality(yellow_official_top_nx, normalized=True)
yellow_official_top_bt = sorted(yellow_official_top_bt.items(), key=lambda x:x[1], reverse=True)
yellow_official_top_bt = DataFrame(data=yellow_official_top_bt)
yellow_official_top_bt.to_csv("ARNOVA Analysis/yellow_official_top_bt_during.csv", sep=',')

yellow_official_bottom_bt = nx.betweenness_centrality(yellow_official_bottom_nx, normalized=True)
yellow_official_bottom_bt = sorted(yellow_official_bottom_bt.items(), key=lambda x:x[1], reverse=True)
yellow_official_bottom_bt = DataFrame(data=yellow_official_bottom_bt)
yellow_official_bottom_b t.to_csv("ARNOVA Analysis/yellow_official_bottom_bt_during.csv", sep=',')

blue_official_top_bt = nx.betweenness_centrality(blue_official_top_nx, normalized=True)
blue_official_top_bt = sorted(blue_official_top_bt.items(), key=lambda x:x[1], reverse=True)
blue_official_top_bt = DataFrame(data=blue_official_top_bt)
blue_official_top_bt.to_csv("ARNOVA Analysis/blue_official_top_bt_during.csv", sep=',')

blue_official_bottom_bt = nx.betweenness_centrality(blue_official_bottom_nx, normalized=True)
blue_official_bottom_bt = sorted(blue_official_bottom_bt.items(), key=lambda x:x[1], reverse=True)
blue_official_bottom_bt = DataFrame(data=blue_official_bottom_bt)
blue_official_bottom_bt.to_csv("ARNOVA Analysis/blue_official_bottom _bt_during.csv", sep=',')

In [86]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(yellow_official_top_cra)
dictionary.save('ARNOVA Analysis/yellow_official_top_cra.dict') 
yellow_official_top_cra_TM = [dictionary.doc2bow(doc) for doc in yellow_official_top_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_top_cra_during.mm', yellow_official_top_cra_TM)

dictionary = corpora.Dictionary(yellow_official_bottom_cra)
dictionary.save('ARNOVA Analysis/yellow_official_bottom_cra.dict') 
yellow_official_bottom_cra_TM = [dictionary.doc2bow(doc) for doc in yellow_official_bottom_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_bottom_cra_during.mm', yellow_official_bottom_cra_TM)

dictionary = corpora.Dictionary(blue_official_top_cra)
dictionary.save('ARNOVA Analysis/blue_official_top_cra.dict') 
blue_official_top_cra_TM = [dictionary.doc2bow(doc) for doc in blue_official_top_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_top_cra_during.mm', blue_official_top_cra_TM)

dictionary = corpora.Dictionary(blue_official_bottom_cra)
dictionary.save('ARNOVA Analysis/blue_official_bottom_cra.dict') 
blue_official_bottom_cra_TM = [dictionary.doc2bow(doc) for doc in blue_official_bottom_cra]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_bottom_cra_during.mm', blue_official_bottom_cra_TM)


In [95]:
dictionary = corpora.Dictionary(yellow_official_top_cut)
dictionary.save('ARNOVA Analysis/yellow_official_top_cut.dict') 
yellow_official_top_cut_TM = [dictionary.doc2bow(doc) for doc in yellow_official_top_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_top_cut_during.mm', yellow_official_top_cut_TM)

dictionary = corpora.Dictionary(yellow_official_bottom_cut)
dictionary.save('ARNOVA Analysis/yellow_official_bottom_cut.dict') 
yellow_official_bottom_cut_TM = [dictionary.doc2bow(doc) for doc in yellow_official_bottom_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/yellow_official_bottom_cut_during.mm', yellow_official_bottom_cut_TM)

dictionary = corpora.Dictionary(blue_official_top_cut)
dictionary.save('ARNOVA Analysis/blue_official_top_cut.dict') 
blue_official_top_cut_TM = [dictionary.doc2bow(doc) for doc in blue_official_top_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_top_cut_during.mm', blue_official_top_cut_TM)

dictionary = corpora.Dictionary(blue_official_bottom_cut)
dictionary.save('ARNOVA Analysis/blue_official_bottom_cut.dict') 
blue_official_bottom_cut_TM = [dictionary.doc2bow(doc) for doc in blue_official_bottom_cut]
corpora.MmCorpus.serialize('ARNOVA Analysis/blue_official_bottom_cut_during.mm', blue_official_bottom_cut_TM)


In [ ]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(yellow_official_top_cra_TM, num_topics=6, id2word = dictionary.load("ARNOVA Analysis/yellow_official_top_cra.dict"), passes=50)

In [98]:
print(ldamodel)

LdaModel(num_terms=19475, num_topics=10, decay=0.5, chunksize=2000)


In [64]:
ldamodel.save('ARNOVA Analysis/model.lda')

In [96]:
print(ldamodel.print_topics(num_topics=5, num_words=5))

[(8, '0.068*警方 + 0.064*人士 + 0.042*記者 + 0.033*市民 + 0.026*現場'), (2, '0.037*學生 + 0.019*立場 + 0.015*同學 + 0.015*香港 + 0.013*明白'), (7, '0.079*政府 + 0.032*香港 + 0.030*社會 + 0.027*民主 + 0.024*市民'), (6, '0.062*旺角 + 0.047*公民 + 0.037*抗命 + 0.022*市民 + 0.020*金鐘'), (3, '0.032*政治 + 0.031*警隊 + 0.014*問題 + 0.011*標語 + 0.010*同事')]


In [106]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, yellow_official_top_cra_TM, dictionary=dictionary.load("ARNOVA Analysis/yellow_official_top_cra.dict"))

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      36.451158        1       1 -0.077852  0.270480
3      24.801911        1       2 -0.074676 -0.216285
2      21.768715        1       3 -0.157210 -0.046455
0      16.978216        1       4  0.309738 -0.007739, topic_info=      Category         Freq Term        Total  loglift  logprob
term                                                           
1894   Default  3540.000000   香港  3540.000000  30.0000  30.0000
7807   Default  2215.000000   警方  2215.000000  29.0000  29.0000
18428  Default  1250.000000   旺角  1250.000000  28.0000  28.0000
678    Default  1026.000000  示威者  1026.000000  27.0000  27.0000
9480   Default  1337.000000   學生  1337.000000  26.0000  26.0000
11941  Default   952.000000   警員   952.000000  25.0000  25.0000
2983   Default  2019.000000   政府  2019.000000  24.0000  24.0000
5707   Default   695.000000   清場   695.000000  23.0000  23.0000
4227   Default   792.000000   記者   792.000000  22.0000  22.0000
5257   Default   730.000000   大家   730.000000  21.0000  21.0000
1405   Default  1048.000000   社會  1048.000000  20.0000  20.0000
14208  Default   953.000000   政治   953.000000  19.0000  19.0000
4063   Default   656.000000   學民   656.000000  18.0000  18.0000
7407   Default   880.000000   民主   880.000000  17.0000  17.0000
8489   Default   594.000000   思潮   594.000000  16.0000  16.0000
17285  Default   591.000000   金鐘   591.000000  15.0000  15.0000
16528  Default   807.000000  梁振英   807.000000  14.0000  14.0000
14887  Default  1263.000000   人士  1263.000000  13.0000  13.0000
5229   Default   454.000000   法庭   454.000000  12.0000  12.0000
18009  Default   495.000000   新聞   495.000000  11.0000  11.0000
14105  Default   701.000000   現場   701.000000  10.0000  10.0000
9021   Default   421.000000   領區   421.000000   9.0000   9.0000
2513   Default   328.000000   警棍   328.000000   8.0000   8.0000
2879   Default   574.000000   問題   574.000000   7.0000   7.0000
10545  Default  1933.000000   市民  1933.000000   6.0000   6.0000
10477  Default  1144.000000   公民  1144.000000   5.0000   5.0000
7738   Default   496.000000   暴力   496.000000   4.0000   4.0000
14913  Default   387.000000  禁制令   387.000000   3.0000   3.0000
10475  Default   374.000000   雨傘   374.000000   2.0000   2.0000
18828  Default   291.000000   同學   291.000000   1.0000   1.0000
...        ...          ...  ...          ...      ...      ...
8542    Topic4   160.184626   學校   160.870483   1.7690  -5.1719
3321    Topic4   147.538007   淚彈   148.230663   1.7686  -5.2541
13194   Topic4   140.228133   突然   140.943631   1.7681  -5.3050
15010   Topic4   108.559423   校長   109.222911   1.7671  -5.5609
11202   Topic4    94.099624   校方    94.760469   1.7662  -5.7039
1577    Topic4    95.407558   獅子    96.099888   1.7660  -5.6901
8305    Topic4    92.721282   父母    93.400765   1.7659  -5.7186
4162    Topic4    89.667577   媽媽    90.348164   1.7657  -5.7521
3209    Topic4    86.561487   傷者    87.240044   1.7654  -5.7874
3863    Topic4    86.254323   武器    86.947446   1.7652  -5.7909
2632    Topic4    80.532436   部隊    81.215918   1.7648  -5.8596
15599   Topic4    82.616414   交代    83.343871   1.7645  -5.8340
11553   Topic4    76.996084   孩子    77.674972   1.7645  -5.9045
4900    Topic4    77.700892   血流    78.388980   1.7644  -5.8954
2639    Topic4    76.888257   家長    77.618071   1.7638  -5.9059
15404   Topic4    70.478914   紀律    71.163764   1.7636  -5.9929
5482    Topic4    68.910394   最低    69.593938   1.7634  -6.0154
16492   Topic4    74.294194   紀錄    75.036883   1.7633  -5.9402
14357   Topic4    65.311416   老師    65.981663   1.7630  -6.0691
9169    Topic4    67.340440  周嘉浩    68.031836   1.7630  -6.0385
6192    Topic4    68.332200   心理    69.038018   1.7630  -6.0238
8661    Topic4   132.043499  中學生   133.583929   1.7616  -5.3651
18304   Topic4    68.174910   學院    68.884868   1.7629  -6.0261
5811    Topic4   131.192957   過程   143.115915   1